## Краулер для Indicator.ru ##

Импортируем все необходимые методы для последующей работы

In [ ]:
import re
import urllib.request
import bs4 as bs
import pandas as pd
import os

Создаем пустой датафрейм. Он пригодится нам в дальнейшем для сортировки данных

In [2]:
ready_df = pd.DataFrame()

## Cкачиваем страницы ##
Функция для скачивания контента по ссылке в виде файла. Аргумент функции - ссылка. Результат работы кода функции - файл в директории "Pages".
Уникальное имя файла генерируется за счет части url (на основе анализа ссылок на статьи в Indicator.ru) за счет замены / на _ и отсекания общей части для всех url.

In [ ]:
def download(url):
    source = urllib.request.urlopen(url).read().decode('utf-8')
    file_name = url.split('https://indicator.ru/article/')[1].replace('/','_')
    fl = open('pages/' + file_name, 'w')
    fl.write(str(source))
    fl.close()

## Записываем тексты статей в отдельные файлы ##

Функция принимает в качестве аргумента текст и имя файла для записи (папка clear_text создана заранее). Данная функция понадобится нам в следующей функции.

In [ ]:
def save_clear_text(text, file_name):
    f = open('clear_text/'+file_name, 'w')
    f.write(text)
    f.close()

## Чистим текст, размечаем данные ##
    Функция принимает на вход путь к файлу (из ранее сгенерированной директории "Pages"). При помощи библиотеки Beautiful Soup на основании предварительного анализа структуры статей сайта находим элементы статьи: 
    - текст статьи
    - заголовок
    - тематическую рубкрику
    - дату (дату берем из ссылки на статью при помощи регулярных выражений)
    - подзаголовок
    - автор (позаголовка и автора может не быть, поэтому в случае их отсутствия пишем None)
    
    Полученные данные добавляем в заранее заготовленный пустой датафрейм; каждая колонка соответсвует вышеперечисленным элементам

In [4]:
def clear(file_path):
    global ready_df #связывает локальную переменную в коде функции с глобальной переменной в общем коде (задали в начале кода)
    with open(file_path) as fl:
        source = fl.read()
    
    soup_article = bs.BeautifulSoup(source, 'lxml')
    
    clear_p = ''    
    for paragraph in soup_article.find_all('div', class_ = 'article__body js-article-body'):
        clear_p = clear_p + paragraph.text
    
    head_title = soup_article.find('div', class_='headline__title').text

    rubric = soup_article.find('div', class_= 'headline__part').text

    date_raw = re.search(r'.*([0-9]{4}_[0-9]{2}_[0-9]{2}).*', file_path)
    date = date_raw.group(1)

    subtitle_raw = soup_article.find('div', class_ = 'headline__subtitle')
    if subtitle_raw == None:
        subtitle = 'None'
    else:
        subtitle = subtitle_raw.text
    
    author_raw = soup_article.find('div', class_ = 'article__author')
    if author_raw == None:
        author = "None"
    else:
        author = author_raw.text.replace('Автор:','').strip()
    
    df = pd.DataFrame([[author,date,subtitle,rubric,head_title,clear_p]], columns=['author','date','subtitle','rubric','head_title','paragraph'])
    
    save_clear_text(clear_p, file_path.replace('pages/',''))
    ready_df = ready_df.append(df)

## Код ##

   К адресу сайта indicator.ru добавляем robots.txt => находим https://indicator.ru/sitemap.xml => находим все ссылки на статьи https://indicator.ru/sitemap-articles.xml.
   При помощи Beuatiful Soup выделяем адреса ссылок.
   Циклом при помощи написанной выше функции скачивания проходим по всем ссылкам статей и скачиваем их в папку "Pages".

In [52]:
sitemap_articles = urllib.request.urlopen('https://indicator.ru/sitemap-articles.xml').read()
soup_links = bs.BeautifulSoup(sitemap_articles, 'lxml')

for article_url in  soup_links.find_all('loc'):
    download(article_url.text)


Проходим по скачанным файлам в папке "Pages", отсеиваем служебные, остальные файлы чистим ранее написанной функцией clear.

In [ ]:
start_path = './pages/'

for path, dirs, files in os.walk(start_path):

    for fname in files:
        if not fname.startswith('.'):
            clear(start_path+fname)

In [13]:
            
ready_df

,author,date,subtitle,rubric,head_title,paragraph
0,Дарья Сапрыкина,2017_01_04,Чем климатологам запомнился 2016 год,Науки о Земле,"Плюсы, минусы, коралловые рифы","Утилизация парникового газа, гибель мозаичнохв..."
0,None,2017_05_22,117 лет назад был выдан патент на пианолу,Технические науки,История науки: окончательный рассказ о механич...,"О том, как механическое пианино прошло путь из..."
0,Алёна Манузина,2016_09_30,135 лет назад родился основатель фирмы «Боинг»,Технические науки,История науки: мечта в металлическом корпусе,"1 октября 1881 года родился человек, фамилию к..."
0,None,2017_08_09,"Астроном Майкл Курц о том, как использовать на...",Астрономия,Наука подобрать: помогает ли наукометрия прини...,Майкл Курц — астрофизик из Гарвардского универ...
0,None,2017_03_18,Трансляция биологического лектория Indicator.R...,Биология,"Три в одном: лекарства, ДНК и магниторецепция....",Информационно-сервисный портал Indicator.Ru со...
0,Яна Комарова,2016_09_29,"Изучен яд улитки-конуса и рецептор, на который...",Биология,Ученые из России создают анальгетики на базе я...,Российские и китайские ученые раскрыли структу...
0,Алексей Паевский,2017_05_27,Отец сосудистого шва,Медицина,Нобелевские лауреаты: Алексис Каррель,"О том, как занятие вышивкой и убийство президе..."
0,Алёна Манузина,2016_10_19,Прошел 171 год с начала работы Русского геогра...,Науки о Земле,"История науки: Новая Гвинея, Тянь-Шань и круго...",19 октября 1845 года в здании Петербургской Ак...
0,Алена Манузина,2016_12_09,48 лет назад была представлена компьютерная мышь,Технические науки,История науки: индикатор X-Y,"9 декабря 1968 года, в первый день Осенней объ..."
0,Дарья Сапрыкина,2017_01_12,Чего ждут и чего боятся научные сотрудники муз...,Гуманитарные науки,Дорога к храму и благие намерения: Исаакиевски...,Исаакиевский собор передают в пользование Церк...


Сохраняем полученный датафрейм в отдельный файл формата csv (comma separated values)

In [11]:
ready_df.to_csv('indicator.csv', sep='\t')